In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#%%
########################################################
######## X-MAC: Trade_off Energy with Delay using GT
########################################################
# Radio subsystem varaible definition

P     = 32.            # Payload [byte]
R     = 31.25          # CC2420 Radio Rate [kbyte/s = Byte/ms]
D     = 8              # number of levels
C     = 5              # neighbors size (connectivity)
N     = C*D**2         # number of nodes
#### BE CAREFUL:  Times are in milliseconds (ms)
Lmax  = 5000.          # Maximal allowed Delay (ms)
Emax  = 1.             # MAximal Energy Budjet (J)

L_pbl = 4.             # preamble length [byte]
L_hdr = 9. + L_pbl     # header length [byte]
L_ack = 9. + L_pbl     # ACK length [byte]
L_ps  = 5. + L_pbl     # preamble strobe length [byte]

Tal  = 0.95            # ack listen period [ms]
Thdr = L_hdr/R         # header transmission duration [ms]
Tack = L_ack/R         # ACK transmission duration [ms]
Tps  = L_ps/R          # preamble strobe transmission duration [ms]
Tcw  = 15*0.62         # Contention window size [ms]
Tcs  = 2.60            # Time [ms] to turn the radio into TX and probe the channel (carrier sense)
Tdata = Thdr + P/R + Tack # data packet transmission duration [ms]

### Sampling frequency
Fs   = 1.0/(60*30*1000)    # e.g. Min traffic rate 1 pkt/half_hour = 1/(60*30*1000) pk/ms

# Sleep period: Parameter Bounds
Tw_max  = 500.       # Maximum Duration of Tw in ms
Tw_min  = 100.       # Minimum Duration of Tw in ms

## Part 1

In [ ]:
intervals= [1,5,10,15,20,25,30] #in minutes
data={}

for i in intervals:
    
    Fs = 1.0/(60*i*1000) #e.g. Min traffic rate 1 pkt/half_hour = 1/(60*30*1000) pk/ms
    print(Fs)
    F_I_worst = Fs * ((D**2)-1)
    F_out_worst = Fs * (D**2)
    F_B_worst = (C-3) * F_out_worst

    alpha1 = Tcs + Tal + (3/2)*Tps * (((Tps+Tal)/2)+Tack+Tdata) * F_B_worst
    alpha2 = F_out_worst/2
    alpha3 = (((Tps+Tal)/2)+Tcs+Tal+Tack+Tdata)*F_out_worst + ((3/2)*Tps + Tack + Tdata) * F_I_worst + (3/4) * Tps * F_B_worst

    beta1 = D/2
    beta2 = D * ((Tcw/2) + Tdata)

    E = lambda T_w : alpha1/T_w + alpha2 * T_w + alpha3
    L = lambda T_w : beta1 * T_w + beta2

    print(alpha1, alpha2, alpha3, beta1, beta2)
    Es = []
    Ls = []
    Tws = []
    for Tw in np.linspace(Tw_min, Tw_max, 10):
        Tws.append(Tw)
        Es.append(E(Tw))
        Ls.append(L(Tw))
    data[i]= {'Tws':Tws, 'Es':Es, 'Ls':Ls}
    
# Plot E(Tw) and L(Tw) as a function of Tw
fig, ax = plt.subplots(2, 1, figsize=(8, 8))
for i in intervals:
    ax[0].plot(data[i]['Tws'], data[i]['Es'], label=f"{i} min")
    ax[1].plot(data[i]['Tws'], data[i]['Ls'], label=f"{i} min")
ax[0].set_xlabel("Tw (ms)")
ax[0].set_ylabel("E (Joules)")
ax[0].set_title("E(Tw) vs Tw")
ax[0].legend()
ax[1].set_xlabel("Tw (ms)")
ax[1].set_ylabel("L (ms)")
ax[1].set_title("L(Tw) vs Tw")
ax[1].legend()
plt.tight_layout()
plt.show()

# Plot E(L) as a function of L
fig, ax = plt.subplots(figsize=(8, 6))
for i in intervals:
    L_vals = np.array(data[i]['Ls'])
    E_vals = np.array(data[i]['Es'])
    ax.plot(L_vals, E_vals, label=f"{i} min")
ax.set_xlabel("L (ms)")
ax.set_ylabel("E (Joules)")
ax.set_title("E(L) vs L")
ax.legend()
plt.show()

In [ ]:
# Create a 7x3 grid of subplots
fig, axs = plt.subplots(7, 3, figsize=(15, 25))

# Plot E(Tw) as a function of Tw in the first column
for i, interval in enumerate(intervals):
    axs[i, 0].plot(data[interval]['Tws'], data[interval]['Es'])
    axs[i, 0].set_xlabel("Tw (ms)")
    axs[i, 0].set_ylabel("E (Joules)")
    axs[i, 0].set_title(f"E(Tw) vs Tw ({interval} min)")
    axs[i, 0].grid(which='both', alpha=0.5)

# Plot E(L) as a function of L in the second column
for i, interval in enumerate(intervals):
    L_vals = np.array(data[interval]['Ls'])
    E_vals = np.array(data[interval]['Es'])
    axs[i, 1].plot(L_vals, E_vals)
    axs[i, 1].set_xlabel("L (ms)")
    axs[i, 1].set_ylabel("E (Joules)")
    axs[i, 1].set_title(f"E(L) vs L ({interval} min)")
    axs[i, 1].grid(which='both', alpha=0.5)
    
# Plot L(Tw) as a function of Tw in the third column
for i, interval in enumerate(intervals):
    axs[i, 2].plot(data[interval]['Tws'], data[interval]['Ls'])
    axs[i, 2].set_xlabel("Tw (ms)")
    axs[i, 2].set_ylabel("L (ms)")
    axs[i, 2].set_title(f"L(Tw) vs Tw ({interval} min)")
    axs[i, 2].grid(which='both', alpha=0.5)


# Adjust subplots spacing and layout
fig.tight_layout(pad=3.0)

# Show the plot
plt.show()

# Part 2

In [ ]:
Fs   = 1.0/(60*30*1000)    # every 30 minutes
Fs

In [ ]:
F_I_worst = Fs * ((D**2)-1)
F_out_worst = Fs * (D**2)
F_B_worst = (C-3) * F_out_worst

alpha1 = Tcs + Tal + (3/2)*Tps * ((Tps+Tal)/2+Tack+Tdata) * F_B_worst
alpha2 = F_out_worst/2
alpha3 = (((Tps+Tal)/2)+Tcs+Tal+Tack+Tdata)*F_out_worst + ((3/2)*Tps + Tack + Tdata) * F_I_worst + (3/4) * Tps * F_B_worst

beta1 = D/2
beta2 = D * ((Tcw/2) + Tdata)

E = lambda T_w : alpha1/T_w + alpha2 * T_w + alpha3
L = lambda T_w : beta1 * T_w + beta2

alpha1, alpha2, alpha3, beta1, beta2

In [ ]:
from gpkit import Variable, Model
import numpy as np

In [ ]:
Tw = Variable('Tw')

## Problem 1

We can use, as suggested, 30 minutes
Calulate the alphas for d=1 and d=D

In [ ]:
E_xmac = alpha1 / Tw + alpha2 * Tw + alpha3
obj = E_xmac

In [ ]:
l_xmac = beta1 * Tw + beta2

In [ ]:
# [100, 5000]
l_max = 5000

In [ ]:
# TODO should be Ceil(Tw/(Tps+Tal)), maybe it's okay
Ttx = Tw /(Tps + Tal) * ((Tps + Tal)/2) + Tack + Tdata

In [ ]:
I_0 = C
Etx_1 = (Tcs + Tal + Ttx) * F_out_worst

In [ ]:
constraints = [
    l_xmac <= l_max,
    Tw >= Tw_min,
    I_0 * Etx_1 <= 1/4,   
]

In [ ]:
prob = Model(obj, constraints)

In [ ]:
prob.solve()

## Problem 2

In [ ]:
l_xmac = beta1 * Tw + beta2
obj = l_xmac

In [ ]:
E_xmac = alpha1 / Tw + alpha2 * Tw + alpha3

In [ ]:
# [0.5, 5]
E_budget = 0.02

In [ ]:
constraints = [
    E_xmac <= E_budget,
    Tw >= Tw_min,
    I_0 * Etx_1 <= 1/4,   
]

In [ ]:
prob = Model(obj, constraints)
prob.solve()

## Problem 3

Nash Barganians solution


In [ ]:
from scipy.optimize import minimize

In [ ]:
# variables E_1, L_1, T_w
# maximize f -> minimize -f
fun = lambda x: - np.log(Emax - x[0]) - np.log(Lmax - x[1]) 

In [ ]:
def cond(x):
    E_1, L_1, T_w = x
    Ttx = T_w /(Tps + Tal) * ((Tps + Tal)/2) + Tack + Tdata
    Etx_1 = (Tcs + Tal + Ttx) * F_out_worst
    return I_0 * Etx_1 - 1/4

In [ ]:
cons = (
    # E_1, L_1, T_w
        {'type': 'ineq', 'fun': lambda x: alpha1 / x[2] + alpha2 * x[2] + alpha3 - Emax},
        {'type': 'ineq', 'fun': lambda x: alpha1 / x[2] + alpha2 * x[2] + alpha3 - x[0]},

        {'type': 'ineq', 'fun': lambda x: beta1 * x[2] + beta2 - Lmax},
        {'type': 'ineq', 'fun': lambda x: beta1 * x[2] + beta2 - x[1]},

        {'type': 'ineq', 'fun': lambda x: Tw_min - x[2]},
        
        {'type': 'ineq', 'fun': cond},
)

In [ ]:
x0 = [0,0,0]
print(minimize(fun, x0, method='SLSQP', constraints=cons))